In [79]:
%matplotlib inline

In [80]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d, fftconvolve
from numpy.fft import rfft2, irfft2

In [81]:
from matplotlib import colors
import matplotlib.animation as animation
from IPython.display import HTML

# Numbers, Arrays and Life

Cellular automata are discrete-space, discrete-time models of spatio-temporal processes. In these models, signal propagation typically involves updating the state of each cell in a grid based on some function of its local neighbors. We will use a toy example, but the techniques you will implement generalize to useful contexts, e.g. spatial statistics or image processing.

We will use John Conway's [Game of Life](https://en.wikipedia.org/wiki/Conway's_Game_of_Life) to practice our manipulation of numerical arrays. In particular, we will see how to use the vectorization, views, convolution and Fourier transform capabilities of `numpy` to solve the same problem in different ways.

See Wikipedia for detailed rules. Here is a summary:

```
Any live cell with fewer than two live neighbors dies, as if caused by under-population.
Any live cell with two or three live neighbors lives on to the next generation.
Any live cell with more than three live neighbors dies, as if by over-population.
Any dead cell with exactly three live neighbors becomes a live cell, as if by reproduction.
```

There are two common ways to count neighbors for cells that are at the edge of the grid. With periodic boundary conditions, the grid wraps around (so it is a torus), that is, the coordinate -1 is mapped to the max_coord, and max_coord + 1 is mapped to 0. Alternatively, just consider any neighbor outside the grid to be equal to zero. For simplicity, we have chosen a life configuration where both strategies give the same results, so you can implement either strategy.

### Initial grid configuration

In [82]:
gun = np.load('gosper_gun.npy')
init_grid = np.zeros((64, 64)).astype('int')
for y, x in gun:
    init_grid[8+x, 8+y] = 1


### Utility function to show animation

In [83]:
def animate(i):
    """Function to display animation for iteration i."""
    global grid
    grid = step(grid)
    im.set_array(grid)
    return im,

**1**. Write a function to play one step in the Game of Life. The function must be named `step` and take a single argument `grid` which is a `numpy` array containing the initial configuration, and return a new `numpy` array containing the updated configuration. Use for loops to implement this first version of the step function.

In [84]:
def step(grid):
    N = [[0,]*(len(grid)) for i in range(len(grid[0]))] # the matrix to store the number of neighbors
    
    #the 4 points in the corner
    N[0][0] = grid[0][1] + grid[1][0] + grid[1][1]
    N[0][len(grid[0])-1]= grid[0][len(grid[0])-2] + grid[1][len(grid[0])-2]+ grid[1][len(grid[0])-1]
    N[len(grid)-1][0]= grid[len(grid)-1][1] + grid[len(grid)-2][0] + grid[len(grid)-2][1] 
    N[len(grid)-1][len(grid[0])-1]= grid[len(grid)-1][len(grid[0])-2]+ grid[len(grid)-2][len(grid[0])-2]+\
    grid[len(grid)-2][len(grid[0])-1]
    
    #the 4 boundaries except 4 corner points
    for i in range(1, len(grid[0])-1):
        N[0][i] = grid[0][i-1] + grid[0][i+1] + grid[1][i] + grid[1][i-1] + grid[1][i+1]
        
    for i in range(1, len(grid[0])-1):
        N[len(grid)-1][i] = grid[len(grid)-1][i-1] + grid[len(grid)-1][i+1] + grid[len(grid)-2][i-1] +\
        grid[len(grid)-2][i+1]+ grid[len(grid)-2][i]
        
    for i in range(1, len(grid)-1):
        N[i][0] = grid[i-1][0] + grid[i+1][0] +grid[i-1][1]+grid[i][1]+grid[i+1][1]
        
    for i in range(1, len(grid)-1):
        N[i][len(grid[0])-1] = grid[i-1][len(grid[0])-1]+ grid[i+1][len(grid[0])-1]+ grid[i-1][len(grid[0])-2] +\
        grid[i+1][len(grid[0])-2]+ grid[i][len(grid[0])-2]
        
    #nonboundary cases
    for i in range(1, len(grid)-1):
        for j in range(1, len(grid[0])-1):
                N[i][j] = grid[i][j-1] + grid[i][j+1] + grid[i-1][j] + grid[i+1][j] + grid[i-1][j-1] +\
                grid[i-1][j+1] + grid[i+1][j+1] + grid[i+1][j-1]
                
    #update the grid 
    for i in range(0, len(grid)):
        for j in range(0, len(grid[0])):
                if grid[i][j] == 0:
                    if N[i][j] == 3:
                         grid[i][j] = 1        
                else:
                    if N[i][j] < 2 or N[i][j] > 3:
                        grid[i][j] = 0
    
    return(grid)

In [85]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
animation.rcParams['animation.writer'] = 'avconv'   
HTML(anim.to_html5_video())

**2**. Rewrite the step function using vectorization. That is, use eight different views of the grid to calculate the neighbor sum instead of double for loops.

In [86]:
# Your solution here
def step(grid):
    N = np.zeros(grid.shape, int)
    # count the number of neighbors for nonboundary cases
    N[1:-1,1:-1] += grid[:-2,:-2]+grid[:-2,1:-1]+grid[:-2, 2:]+\
                grid[1:-1, :-2] +grid[1:-1, 2:]+\
                grid[2:, :-2] +grid[2:, 1:-1] +grid[2:,2:]
    # boundary cases without 4 corner points 
    N[0,1:-1] += grid[0,:-2]+grid[0,2:]+grid[1,:-2]+grid[1,1:-1]+grid[1,2:]
    N[grid.shape[0]-1, 1:-1] += grid[grid.shape[0]-1, :-2]+grid[grid.shape[0]-1,2:]+grid[grid.shape[0]-2,:-2]+\
                                grid[grid.shape[0]-2,1:-1]+grid[grid.shape[0]-2,2:]
    N[1:-1,0] += grid[:-2,0]+grid[2:,0]+grid[:-2, 1]+grid[1:-1,1]+grid[2:,1]
    N[1:-1,grid.shape[1]-1] += grid[:-2, grid.shape[1]-1]+grid[2:, grid.shape[1]-1]+ grid[:-2, grid.shape[1]-2]+\
                               grid[1:-1, grid.shape[1]-2]+grid[2:, grid.shape[1]-2]
    # 4 corner points
    N[0][0] = grid[0,1] + grid[1,0] + grid[1,1]
    N[0,len(grid[0])-1]= grid[0,len(grid[0])-2] + grid[1,len(grid[0])-2]+ grid[1,len(grid[0])-1]
    N[len(grid)-1,0]= grid[len(grid)-1,1] + grid[len(grid)-2,0] + grid[len(grid)-2,1] 
    N[len(grid)-1,len(grid[0])-1]= grid[len(grid)-1,len(grid[0])-2]+ grid[len(grid)-2,len(grid[0])-2]+\
                                    grid[len(grid)-2,len(grid[0])-1]
    #update grid
    birth = (N==3) & (grid[:,:]==0)
    survive = ((N==2) | (N==3)) & (grid[:,:]==1)
    grid[...] = 0
    grid[:,:][birth | survive] = 1   
    return grid


In [87]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
animation.rcParams['animation.writer'] = 'avconv'   
HTML(anim.to_html5_video())

**3a**. A discrete 2D convolution generates a weighted sum of a 2D grid, with the weights given by a 2D kernel. For example, the kernel

```
0 1 0
1 1 1
0 1 0
```

would result in summing the current location and the N, E, S, W neighbors.

Use the `convolve2d` function from `scipy.signal` (already imported for you in this notebook) to implement the 3rd version of the `step` function with a suitable kernel and with `mode=same` to preserve the grid size across iterations.

In [73]:
# Your solution here
def step(grid):
    kernel = np.array([[1,1,1],
                      [1,0,1],
                      [1,1,1]])
    N = convolve2d(grid, kernel, mode='same',boundary = 'fill')
    
    #update grid
    birth = (N==3) & (grid[:,:]==0)
    survive = ((N==2) | (N==3)) & (grid[:,:]==1)
    grid[...] = 0
    grid[:,:][birth | survive] = 1   
    return(grid)

In [74]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
animation.rcParams['animation.writer'] = 'avconv'
HTML(anim.to_html5_video())

**3b**. One way to multiply two numbers that you are familiar with is to move to the log domain where addition is equivalent to multiplication. For example, instead of calculating $100 \times 1000$, we can calculate $\exp(\log(100) + \log(1000))$ to get the same result to roundoff error. In the same way, a convolution is equivalent to multiplication in the Fourier domain.  

Implement the fourth version of the `step` function using the `fftconvolve` function for real Fast Fourier Transforms from the `scipy.signal` package. Because of rounding errors, you need to round the results returned by `fftconvolve`.

In [75]:
# Your solution here
def step(grid):
    array = np.array([[1,1,1],
                      [1,0,1],
                      [1,1,1]])
    N = fftconvolve(grid, array, mode='same')
    N = np.around(N,decimals =0)
    #update grid
    birth = (N==3) & (grid[::]==0)
    survive = ((N==2) | (N==3)) & (grid[::]==1)
    grid[...] = 0
    grid[::][birth | survive] = 1   
    return(grid)

In [76]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
animation.rcParams['animation.writer'] = 'avconv'
HTML(anim.to_html5_video())

**4**. Modify 3a to model the [Forest Fire model](https://en.wikipedia.org/wiki/Forest-fire_model) instead of the Game of Life, with `f=0.001` and `p=0.1`.

In [77]:
# Your solution here
def step(grid):
    f =0.001
    p =0.1
    #calculate the index of the neighbors
    index = np.array([(-1,-1),(-1,0),(-1,1),
                      (0, -1),(0,0),(0,1),
                      (1,-1),(1,0),(1,1)])
    #define the status
    EMPTY, TREE, FIRE = 0, 1, 2
    
    #update grid
    grid1 = np.zeros(grid.shape)
    for i in range(1, grid.shape[0]-1):
        for j in range(1, grid.shape[1]-1):
            if grid[i,j] == 0 and np.random.random() <= p:
                    grid1[i,j] = 1
            elif grid[i,j] == 1 and np.random.random() <= f:
                    grid1[i,j] = 2
            elif grid[i,j] == 1:
                grid1[i,j] = 1
                for d in index:
                    if grid[i+d[0],j+d[1]] == 2:
                        grid1[i,j] = 2
                        break
            elif grid[i,j] == 2:
                grid1[i,j] = 0
                                
    return(grid1)

In [78]:
cmap = colors.ListedColormap(['black', 'green', 'red'])

fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap=cmap, vmax=2)
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=100, interval=50, blit=True)
animation.rcParams['animation.writer'] = 'avconv'
HTML(anim.to_html5_video())